In [2]:
# Data set files provided in Task 6
hygiene_text_path= "./Hygiene/hygiene.dat"
hygiene_labels_path= "./Hygiene/hygiene.dat.labels"
hygiene_additional_path= "./Hygiene/hygiene.dat.additional"

In [3]:
# Ignore the warning
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Import the required libraries
import pandas as pd
import numpy as np
import multiprocessing
import gensim
import nltk

from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin


from gensim.parsing.preprocessing import strip_tags, strip_punctuation, strip_numeric, strip_short
from gensim.parsing.preprocessing import strip_multiple_whitespaces, strip_non_alphanum, remove_stopwords, stem_text
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from UtilWordEmbedding import MeanEmbeddingVectorizer

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.feature_extraction import DictVectorizer

from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn import svm
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, GridSearchCV
from tqdm import tqdm


from nltk.corpus import stopwords 
STOP_WORDS = set(stopwords.words('english'))


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
SEED=26

## 1. Data Preprpcessing
Preprocessing Steps:
* tokenization and cleaning
* stopword removal
* stemming and lemmatization

In [5]:
# tokenize and preprocess
# https://radimrehurek.com/gensim/parsing/preprocessing.html
FILTERS_LIST = [lambda x: x.lower(), # lowercase  
                strip_tags, # remove tags
                strip_punctuation, # replace punctuation characters with spaces
                strip_multiple_whitespaces, # remove repeating whitespaces
                # strip_numeric, # remove numbers
                gensim.parsing.preprocessing.remove_stopwords, # remove stopwords
                strip_short, # remove words less than minsize=3 characters long]
                stem_text]
def preprocess(text):
    """
    strip_tags, strip_punctuation, strip_multiple_whitespaces, strip_numeric, 
    """
    result_stemmed = []
    for token in gensim.parsing.preprocessing.preprocess_string(text, FILTERS_LIST):
        result_stemmed.append(WordNetLemmatizer().lemmatize(token))
    return result_stemmed

In [6]:
%%time
texts = []
preprocessed_texts = []

with open(hygiene_text_path) as f:
    texts = f.readlines()
    
for _text in tqdm(texts):
    result_stemmed = preprocess(_text)
    preprocessed_texts.append(result_stemmed)
    
all_preprocessed_texts = [" ".join(_text) for _text in preprocessed_texts]

100%|███████████████████████████████████████████████████████████████████████████| 13299/13299 [02:02<00:00, 108.62it/s]


Wall time: 2min 3s


In [7]:
N = 546

# labels 
with open(hygiene_labels_path, 'r') as f:
    labels = [l.rstrip() for l in f]


df = pd.DataFrame({"label":labels, "text": texts, 
                   "preprocessed_texts": all_preprocessed_texts,
                   "tokenized_texts": preprocessed_texts})
hygiene_additional = pd.read_csv(hygiene_additional_path,  
                                 names=["cuisines_offered", "zipcode", "num_reviews", "avg_rating"],
                                 dtype={"cuisines_offered": str, 
                                        "zipcode": str,
                                        "num_reviews": str})
df = df.join(hygiene_additional)
df['avg_rating'] = df['avg_rating'].apply(lambda x: str(int(round(x, 0))))

print(df.info())
display(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13299 entries, 0 to 13298
Data columns (total 8 columns):
label                 13299 non-null object
text                  13299 non-null object
preprocessed_texts    13299 non-null object
tokenized_texts       13299 non-null object
cuisines_offered      13299 non-null object
zipcode               13299 non-null object
num_reviews           13299 non-null object
avg_rating            13299 non-null object
dtypes: object(8)
memory usage: 831.3+ KB
None


,label,text,preprocessed_texts,tokenized_texts,cuisines_offered,zipcode,num_reviews,avg_rating
0,1,"The baguettes and rolls are excellent, and alt...",baguett roll excel haven tri excit dozen plu t...,"[baguett, roll, excel, haven, tri, excit, doze...","['Vietnamese', 'Sandwiches', 'Restaurants']",98118,4,4
1,1,I live up the street from Betty. &#160;When my...,live street betti 160 sister town spring break...,"[live, street, betti, 160, sister, town, sprin...","['American (New)', 'Restaurants']",98109,21,4
2,1,I'm worried about how I will review this place...,worri review place strongli think bad night pl...,"[worri, review, place, strongli, think, bad, n...","['Mexican', 'Restaurants']",98103,14,3
3,0,Why can't you access them on Google street vie...,access googl street view like medina yarrow po...,"[access, googl, street, view, like, medina, ya...","['Mexican', 'Tex-Mex', 'Restaurants']",98112,42,4
4,0,Things to like about this place: homemade guac...,thing like place homemad guacamol varieti tast...,"[thing, like, place, homemad, guacamol, variet...","['Mexican', 'Restaurants']",98102,12,3


In [8]:
%%time
train_df = df[df["label"] != "[None]"]
test_df = df[df["label"] == "[None]"]

additional_feats = ["cuisines_offered", "zipcode", "num_reviews", "avg_rating"]

train = train_df[["text"] + additional_feats]
train_preprocessed = train_df[["preprocessed_texts"] + additional_feats]
train_tokenized = train_df[["tokenized_texts"] + additional_feats]
train_labels = train_df["label"].astype(int) # needed by sklearn

test = test_df[["text"] + additional_feats]
test_preprocessed = test_df[["preprocessed_texts"] + additional_feats]
test_tokenized = test_df[["tokenized_texts"] + additional_feats]
test_labels = test_df["label"]

print(train.shape, train_preprocessed.shape, train_tokenized.shape, train_labels.shape)
print(test.shape, test_preprocessed.shape, test_tokenized.shape, test_labels.shape)
print(train.dtypes, train_preprocessed.dtypes, train_tokenized.dtypes)

(546, 5) (546, 5) (546, 5) (546,)
(12753, 5) (12753, 5) (12753, 5) (12753,)
text                object
cuisines_offered    object
zipcode             object
num_reviews         object
avg_rating          object
dtype: object preprocessed_texts    object
cuisines_offered      object
zipcode               object
num_reviews           object
avg_rating            object
dtype: object tokenized_texts     object
cuisines_offered    object
zipcode             object
num_reviews         object
avg_rating          object
dtype: object
Wall time: 21.9 ms


In [9]:
display(train.head())
display(train_preprocessed.head())
display(train_tokenized.head())

,text,cuisines_offered,zipcode,num_reviews,avg_rating
0,"The baguettes and rolls are excellent, and alt...","['Vietnamese', 'Sandwiches', 'Restaurants']",98118,4,4
1,I live up the street from Betty. &#160;When my...,"['American (New)', 'Restaurants']",98109,21,4
2,I'm worried about how I will review this place...,"['Mexican', 'Restaurants']",98103,14,3
3,Why can't you access them on Google street vie...,"['Mexican', 'Tex-Mex', 'Restaurants']",98112,42,4
4,Things to like about this place: homemade guac...,"['Mexican', 'Restaurants']",98102,12,3


,preprocessed_texts,cuisines_offered,zipcode,num_reviews,avg_rating
0,baguett roll excel haven tri excit dozen plu t...,"['Vietnamese', 'Sandwiches', 'Restaurants']",98118,4,4
1,live street betti 160 sister town spring break...,"['American (New)', 'Restaurants']",98109,21,4
2,worri review place strongli think bad night pl...,"['Mexican', 'Restaurants']",98103,14,3
3,access googl street view like medina yarrow po...,"['Mexican', 'Tex-Mex', 'Restaurants']",98112,42,4
4,thing like place homemad guacamol varieti tast...,"['Mexican', 'Restaurants']",98102,12,3


,tokenized_texts,cuisines_offered,zipcode,num_reviews,avg_rating
0,"[baguett, roll, excel, haven, tri, excit, doze...","['Vietnamese', 'Sandwiches', 'Restaurants']",98118,4,4
1,"[live, street, betti, 160, sister, town, sprin...","['American (New)', 'Restaurants']",98109,21,4
2,"[worri, review, place, strongli, think, bad, n...","['Mexican', 'Restaurants']",98103,14,3
3,"[access, googl, street, view, like, medina, ya...","['Mexican', 'Tex-Mex', 'Restaurants']",98112,42,4
4,"[thing, like, place, homemad, guacamol, variet...","['Mexican', 'Restaurants']",98102,12,3


## 2. Applied Methods

Models:  
* Naive Bayes
* SVM
* Logistic Regression
* Random Forest
* XGBoost


In [11]:
%%time
from sklearn import preprocessing

pipeline = Pipeline([
    ('preprocess', ColumnTransformer(
        [('cuisines_offered', CountVectorizer(min_df=10), 'cuisines_offered'),
         ('zipcode', OneHotEncoder(dtype='int', handle_unknown='ignore'), ['zipcode']),
         ('num_reviews', CountVectorizer(max_df=7, token_pattern='\d+'), 'num_reviews'),
         ('avg_rating', OneHotEncoder(dtype='int', handle_unknown='ignore'), ['avg_rating']),
         ('text', TfidfVectorizer(
                    stop_words='english',
                    strip_accents='unicode',
                    min_df=3,
                    max_df=0.5,
                    ngram_range=(1, 3),
                    max_features=500), 'preprocessed_texts')],
        remainder='passthrough',
    )),
    ('clf', MultinomialNB())
], verbose=False)


scores = cross_val_score(pipeline, train_preprocessed, train_labels, cv=5, scoring= 'f1_macro')
print(scores)
print("Average F1-Score: %0.5f" % np.average(scores))

[0.62650104 0.71815853 0.67175066 0.69420849 0.60238429]
Average F1-Score: 0.66260
Wall time: 12.7 s


In [12]:
%%time
from sklearn import preprocessing

pipeline = Pipeline([
    ('preprocess', ColumnTransformer(
        [('cuisines_offered', CountVectorizer(min_df=10), 'cuisines_offered'),
         ('zipcode', OneHotEncoder(dtype='int', handle_unknown='ignore'), ['zipcode']),
         ('num_reviews', CountVectorizer(max_df=7, token_pattern='\d+'), 'num_reviews'),
         ('avg_rating', OneHotEncoder(dtype='int', handle_unknown='ignore'), ['avg_rating']),
         ('text', TfidfVectorizer(
                    stop_words='english',
                    strip_accents='unicode',
                    min_df=3,
                    max_df=0.5,
                    ngram_range=(1, 3),
                    max_features=500), 'text')],
        remainder='passthrough',
    )),
    ('clf', MultinomialNB())
], verbose=False)


scores = cross_val_score(pipeline, train, train_labels, cv=5, scoring= 'f1')
print(scores)
print("Average F1-Score: %0.5f" % np.average(scores))

[0.62857143 0.71559633 0.68965517 0.66666667 0.52747253]
Average F1-Score: 0.64559
Wall time: 15.2 s


In [13]:
#print(len(df['num_reviews'].value_counts()))
#df['num_reviews'].value_counts()

In [14]:
#print(len(df['cuisines_offered'].value_counts()))
#df['cuisines_offered'].value_counts()

In [15]:
print(len(df['avg_rating'].value_counts()))
print(len(df['zipcode'].value_counts()))

5
30


### Create Function for Testing

In [16]:
%%time
def test_classifier(clf, X, y, vectorizer, text_col='text'):
    pipeline = Pipeline([
        ('union', ColumnTransformer(
        [('cuisines_offered', CountVectorizer(min_df=10), 'cuisines_offered'),
         ('zipcode', OneHotEncoder(dtype='int', handle_unknown='ignore'), ['zipcode']),
         ('num_reviews', CountVectorizer(max_df=7, token_pattern='\d+'), 'num_reviews'),
         ('avg_rating', OneHotEncoder(dtype='int', handle_unknown='ignore'), ['avg_rating']),
         ('text', vectorizer, text_col)],
        remainder='passthrough',
    )),
        ('clf', clf)
    ], verbose=False)
    scores = cross_val_score(pipeline, X, y, cv=5, scoring= 'f1_macro')
    print(clf)
    print(scores)
    cv_score = np.average(scores)
    return cv_score

Wall time: 0 ns


In [17]:
classifiers = {
    'Naive Bayes': MultinomialNB(),
    'Support Vector Machine': svm.SVC(),
    'Logistic Regression': LogisticRegression(),
    'Random Forest': RandomForestClassifier(random_state=SEED, n_estimators=500, n_jobs=-1),
    #'Gradient Boosting': GradientBoostingClassifier()
    'XGBoost': XGBClassifier(n_estimators=500, 
                            max_depth=5, 
                            learning_rate=0.2, 
                            objective='binary:logistic',
                            scale_pos_weight=2,
                            n_jobs=-1,
                            random_state=SEED)
}

tfidf = TfidfVectorizer(
                    stop_words='english',
                    strip_accents='unicode',
                    min_df=3,
                    max_df=0.5,
                    ngram_range=(1, 3),
                    max_features=500)
bow = CountVectorizer(
                stop_words=STOP_WORDS,
                strip_accents='unicode',
                min_df=15,
                max_df=0.5,
                ngram_range=(1, 3))

### 2.1 BOW - No Preprocessing

In [18]:
%%time
for clf_name, clf in classifiers.items():
    cv_score = test_classifier(clf, train, train_labels, 
                               vectorizer=bow, text_col='text')
    print('{}: {}'.format(clf_name, cv_score))

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
[0.59880637 0.71797205 0.69080688 0.65714286 0.60625   ]
Naive Bayes: 0.6541956293640896
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)
[0.61117002 0.59430483 0.56227246 0.63291592 0.5459217 ]
Support Vector Machine: 0.5893169873985258
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)
[0.56349206 0.57678153 0.59947037 0.62950257 0.62642684]
Logistic Regression: 0.5991346769294893
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            

### 2.2 BOW - Preprocessing

In [19]:
%%time
for clf_name, clf in classifiers.items():
    cv_score = test_classifier(clf, train_preprocessed, train_labels, 
                               vectorizer=bow, text_col='preprocessed_texts')
    print('{}: {}'.format(clf_name, cv_score))

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
[0.60750145 0.73634185 0.68179188 0.63861004 0.60774818]
Naive Bayes: 0.6543986809702782
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)
[0.56468209 0.58632479 0.56463158 0.6082456  0.52574045]
Support Vector Machine: 0.5699248992795691
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)
[0.55450864 0.53323904 0.60879993 0.65714286 0.6462069 ]
Logistic Regression: 0.5999794725593286
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            

### 2.3 TFIDF - No Preprocessing

In [20]:
%%time
for clf_name, clf in classifiers.items():
    cv_score = test_classifier(clf, train, train_labels, 
                               vectorizer=tfidf, text_col='text')
    print('{}: {}'.format(clf_name, cv_score))

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
[0.6447205  0.71815853 0.67175066 0.67567568 0.59173626]
Naive Bayes: 0.6604083249868596
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)
[0.68179188 0.76166667 0.68179188 0.60774818 0.55563506]
Support Vector Machine: 0.6577267361102672
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)
[0.63588216 0.70909091 0.63527851 0.62950257 0.63636364]
Logistic Regression: 0.6492235582335915
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            

### 2.3 TFIDF - Preprocessing

In [21]:
%%time
for clf_name, clf in classifiers.items():
    cv_score = test_classifier(clf, train_preprocessed, train_labels, 
                               vectorizer=tfidf, text_col='preprocessed_texts')
    print('{}: {}'.format(clf_name, cv_score))

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
[0.62650104 0.71815853 0.67175066 0.69420849 0.60238429]
Naive Bayes: 0.662600601951647
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)
[0.67272727 0.75289126 0.67272727 0.61771562 0.56696071]
Support Vector Machine: 0.6566044248751868
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)
[0.65408805 0.72718254 0.67229394 0.61057692 0.61478904]
Logistic Regression: 0.6557860988379179
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             

## 3. Submission

In [22]:
def create_submission(y_pred, filepath):
    with open(filepath, 'w') as f:
        f.write('sp2\n')
        for label in y_pred:
            f.write(str(int(label)) + '\n')

In [23]:
%%time
pipeline = Pipeline([
    ('preprocess', ColumnTransformer(
        [('cuisines_offered', CountVectorizer(min_df=10), 'cuisines_offered'),
         ('zipcode', OneHotEncoder(dtype='int', handle_unknown='ignore'), ['zipcode']),
         ('num_reviews', CountVectorizer(max_df=7, token_pattern='\d+'), 'num_reviews'),
         ('avg_rating', OneHotEncoder(dtype='int', handle_unknown='ignore'), ['avg_rating']),
         ('text', TfidfVectorizer(
                    stop_words='english',
                    strip_accents='unicode',
                    min_df=3,
                    max_df=0.5,
                    ngram_range=(1, 3),
                    max_features=500), 'preprocessed_texts')],
        remainder='passthrough',
    )),
    ('clf', MultinomialNB())
], verbose=False)

pipeline.fit(train_preprocessed, train_labels)
y_pred = pipeline.predict(test_preprocessed)
# score = metrics.f1_score(y_test, y_pred)
# scores = cross_val_score(pipeline, train_preprocessed, train_labels, cv=5, scoring= 'f1_macro')
# print(scores)
# print("Average F1-Score: %0.5f" % np.average(scores))

Wall time: 20.8 s


In [24]:
submit_path ='./submissions/submission_task6.txt'
create_submission(y_pred, submit_path)

In [25]:
submit_path

'./submissions/submission_task6.txt'

In [26]:
!python submit.py sp2 {submit_path}

Submission completed successfully!
